In [8]:
pip install cybersource-rest-client-python

## Carga de Librerias

In [9]:
import requests
import pandas as pd
from io import StringIO
from google.cloud import bigquery, storage
from datetime import datetime, timedelta
import pandas as pd
import requests
from io import StringIO
from google.cloud import bigquery, storage
import pyarrow as pa
import pyarrow.parquet as pq
from pandas.errors import EmptyDataError
import gc
import time
import pandas as pd
import os
import gc
from datetime import datetime, timedelta
from google.cloud import storage
from google.cloud import bigquery
from io import StringIO
from google.api_core.exceptions import GoogleAPICallError
from pandas.errors import EmptyDataError
from CyberSource import *
from pathlib import Path
import os
import json
from importlib.machinery import SourceFileLoader
from CyberSource.api.report_downloads_api import ReportDownloadsApi
import os
import pandas as pd

## Configuracion Inicial

In [10]:
# Crear carpeta si no existe

folder_path = '/content/data'
os.makedirs(folder_path, exist_ok=True)

# Código fuente de Configuration.py
code = '''
import os
from CyberSource.logging.log_configuration import LogConfiguration

class Configuration:
    def __init__(self):
        self.authentication_type ="http_signature"
        self.merchantid = "izipay_master_acct"
        self.run_environment = "api.cybersource.com"
        self.request_json_path = ""
        # JWT PARAMETERS
        self.key_alias = "izipay_master_acct"
        self.key_pass = "izipay_master_acct"
        self.key_file_name = "izipay_master_acct"
        self.keys_directory = os.path.join(os.getcwd(), "resources")
        # HTTP PARAMETERS
        self.merchant_keyid = "98c3b49d-0dc0-4a35-933c-0037d1789462"
        self.merchant_secretkey = "/mxDSoyXaym+HDoVtqPm8zbdQ3Poos3CO3ZHlIEHOxc=="
        # META KEY PARAMETERS
        self.use_metakey = False
        self.portfolio_id = ''
        # CONNECTION TIMEOUT PARAMETER
        self.timeout = 1000
        # LOG PARAMETERS
        self.enable_log = True
        self.log_file_name = "cybs"
        self.log_maximum_size = 10487560
        self.log_directory = os.path.join(os.getcwd(), "Logs")
        self.log_level = "Debug"
        self.enable_masking = True
        self.log_format = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        self.log_date_format = "%Y-%m-%d %H:%M:%S"
        self.useMLEGlobally = False

    def get_configuration(self):
        configuration_dictionary = {}
        configuration_dictionary["authentication_type"] = self.authentication_type
        configuration_dictionary["merchantid"] = self.merchantid
        configuration_dictionary["run_environment"] = self.run_environment
        configuration_dictionary["request_json_path"] = self.request_json_path
        configuration_dictionary["key_alias"] = self.key_alias
        configuration_dictionary["key_password"] = self.key_pass
        configuration_dictionary["key_file_name"] = self.key_file_name
        configuration_dictionary["keys_directory"] = self.keys_directory
        configuration_dictionary["merchant_keyid"] = self.merchant_keyid
        configuration_dictionary["merchant_secretkey"] = self.merchant_secretkey
        configuration_dictionary["use_metakey"] = self.use_metakey
        configuration_dictionary["portfolio_id"] = self.portfolio_id
        configuration_dictionary["timeout"] = self.timeout
        configuration_dictionary['useMLEGlobally'] = self.useMLEGlobally

        log_config = LogConfiguration()
        log_config.set_enable_log(self.enable_log)
        log_config.set_log_directory(self.log_directory)
        log_config.set_log_file_name(self.log_file_name)
        log_config.set_log_maximum_size(self.log_maximum_size)
        log_config.set_log_level(self.log_level)
        log_config.set_enable_masking(self.enable_masking)
        log_config.set_log_format(self.log_format)
        log_config.set_log_date_format(self.log_date_format)

        configuration_dictionary["log_config"] = log_config
        return configuration_dictionary
'''

# Guardar archivo
file_path = os.path.join(folder_path, 'Configuration.py')
with open(file_path, 'w') as f:
    f.write(code.strip())

print(f"Archivo creado exitosamente en: {file_path}")


Archivo creado exitosamente en: /content/data/Configuration.py


In [11]:
# Crear carpeta si no existe
resources_path = '/content/resources'
os.makedirs(resources_path, exist_ok=True)

# Crear CSV vacío
csv_path = os.path.join(resources_path, 'download_report.csv')
pd.DataFrame().to_csv(csv_path, index=False)

print(f"Archivo CSV vacío creado en: {csv_path}")

Archivo CSV vacío creado en: /content/resources/download_report.csv


## Funciones Iniciales

### Cargar Datos Cybersource

In [21]:
from CyberSource import *
from pathlib import Path
import os
import json
from importlib.machinery import SourceFileLoader
from CyberSource.api.report_downloads_api import ReportDownloadsApi

config_file = os.path.join(os.getcwd(), "data", "Configuration.py")
configuration = SourceFileLoader("module.name", config_file).load_module()


def del_none(d):
    for key, value in list(d.items()):
        if value is None:
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d

def download_report(process_date):
    organizationId = "izipay_master_acct"
    reportDate = process_date
    reportName = "TRR_daily"

    try:
        config_obj = configuration.Configuration()
        client_config = config_obj.get_configuration()
        api_instance = ReportDownloadsApi(client_config)
        api_instance.api_client.download_file_path = os.path.join(os.getcwd(), "resources", "download_report.csv")
        status, headers = api_instance.download_report(reportDate, reportName, organization_id=organizationId)

        print("Download Status : ", status)
        print("Response Headers : ", headers)

        print("Response downloaded at the location : " + api_instance.api_client.download_file_path)
        #write_log_audit(status)
    except Exception as e:
        #write_log_audit(e.status if hasattr(e, 'status') else 999)
        print("\nException when calling ReportDownloadsApi->download_report: %s\n" % e)

#def write_log_audit(status):
#    print(f"[Sample Code Testing] [{Path(__file__).stem}] {status}")

### Guardar CSV a Pandas

In [22]:
import pandas as pd

def load_cyber_report(csv_path='/content/resources/download_report.csv'):
    try:
        # Intentar leer el archivo, saltando la primera fila y forzando todo a string
        df = pd.read_csv(csv_path, dtype=str, skiprows=1)

        # Verificar si el DataFrame está vacío o tiene columnas mal formateadas
        if df.empty or any('unnamed' in col.lower() for col in df.columns):
            print("❌ Error en la carga: el archivo está vacío o tiene columnas inválidas ('Unnamed').")
            df_cyber = pd.DataFrame()
        else:
            print("✅ Archivo cargado correctamente.")
            print("Columnas:", df.columns.tolist())
            print("Número de filas:", len(df))
            df_cyber = df

    except Exception as e:
        print(f"❌ Error al intentar leer el archivo: {e}")
        df_cyber = pd.DataFrame()

    return df_cyber


### Guardar en GCS

In [24]:
def fn_ingesta(fecha, var_bucket_name, var_project_id, var_dataset_id, df_cyber):
    from datetime import datetime, timedelta
    import gc
    from google.cloud import storage

    fecha0 = datetime.strptime(fecha, "%Y-%m-%d")
    fecha_d_1 = fecha0 - timedelta(days=1)
    fecha_d_1yyyymmdd = fecha_d_1.strftime("%Y%m%d")

    try:
        df = df_cyber.copy()

        # Eliminar columnas sin cabecera
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        df = df.astype(str).replace('nan', None)

        if df.empty or df.isnull().all().all():
            print(f"⚠️ Archivo creado para {fecha}, pero está vacío. No se cargará.")
        else:
            # Guardar en Parquet
            file_name = f"TRR_daily_{fecha_d_1yyyymmdd}.parquet"
            df.to_parquet(file_name, index=False)

            # Subir a GCS
            storage_client = storage.Client()
            bucket = storage_client.bucket(var_bucket_name)
            destination_blob_name = f"Raw/Cybersource/Transaccional/TRR_daily/{fecha_d_1yyyymmdd[:4]}/{fecha_d_1yyyymmdd[4:6]}/{file_name}"
            blob = bucket.blob(destination_blob_name)
            blob.upload_from_filename(file_name)

            print(f"✅ Archivo '{file_name}' guardado en Cloud Storage en '{destination_blob_name}'")

        del df
        gc.collect()

    except Exception as e:
        print(f"❌ Error durante la ingesta para {fecha}: {e}")


## Variables

In [15]:
# Obtener la fecha actual
current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
var_bucket_name = 'dev-ingesta-izipay-fx32'
var_project_id  = 'dev-izipay-data-storage'
var_dataset_id = ''
var_file = ''
var_source =''
fecha =  formatted_date

## Ejecutar funciones de Carga

In [16]:
## Archivos Cybersource :

if __name__ == "__main__":
    download_report(fecha)

INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ff89b03790>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-05-12&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 15:27:22 GMT
INFO:Authorization:Host: 

HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_e1eb3e38-e70d-4c4d-acda-af574c2c5fa5-15036-2090', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:27:23 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 15:27:23 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '688e2638-ca32-4df2-a481-dc69f748ba42'})
Download Status :  200
Response Headers :  HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_e1eb3e38-e70d-4c4d-acda-af574c2c5fa5-15036-2090', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:27:23 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 M

In [17]:
# Guardar en pandas Dataframe :
df_cyber = load_cyber_report()

✅ Archivo cargado correctamente.
Columnas: ['RequestID', 'RequestDate', 'MerchantID', 'LocalizedRequestDate', 'ApplicationName', 'OverallRcode', 'OverallReasonCode', 'OverallRflag', 'RCode', 'ReasonCode', 'RFlag', 'RMsg', 'BankAddress', 'BankBranchCode', 'BankCity', 'BankCountry', 'BankName', 'BankSwiftCode', 'BillTo_Address1', 'BillTo_Address2', 'BillTo_City', 'BillTo_CompanyName', 'BillTo_Company_tax_id', 'BillTo_Country', 'BillTo_CustomerID', 'BillTo_Email', 'BillTo_FirstName', 'BillTo_HostName', 'BillTo_IPAddress', 'BillTo_LastName', 'BillTo_MiddleName', 'BillTo_NameSuffix', 'BillTo_PersonalID', 'BillTo_Phone', 'BillTo_State', 'BillTo_Title', 'BillTo_UserName', 'BillTo_Zip', 'check_account_encoderID', 'BankTransitNumber', 'SecCode', 'DeviceID', 'BankCheckDigit', 'IbanIndicator', 'GiftCard_CurrentBalance', 'GiftCard_PreviousBalance', 'InvoiceHeader_CustomData_1', 'FulfillmentType', 'InvoiceNumber', 'MerchantProductSKU', 'LineItemNumber', 'ProductCode', 'ProductName', 'Quantity', 'Ta

In [25]:
## Ingestar Datos :
fn_ingesta(fecha,var_bucket_name,var_project_id,var_dataset_id,df_cyber)

✅ Archivo 'TRR_daily20250511.parquet' guardado en Cloud Storage en 'Raw/Cybersource/Transaccional/TRR_daily/2025/05/TRR_daily20250511.parquet'


## Reprocesos :


In [ ]:
from datetime import datetime, timedelta

# Define tus fechas de inicio y fin en formato YYYY-MM-DD
fecha_ini = "2025-01-24"
fecha_fin = "2025-05-11"

# Variables fijas
var_bucket_name = 'dev-ingesta-izipay-fx32'
var_project_id  = 'dev-izipay-data-storage'
var_dataset_id = ''

# Bucle por cada fecha del rango
fecha_actual = datetime.strptime(fecha_ini, "%Y-%m-%d")
fecha_limite = datetime.strptime(fecha_fin, "%Y-%m-%d")

while fecha_actual <= fecha_limite:
    fecha_str = fecha_actual.strftime("%Y-%m-%d")

    print(f"\n📆 Procesando fecha: {fecha_str}")

    # 1. Descargar reporte
    download_report(fecha_str)

    # 2. Leer CSV
    df_cyber = load_cyber_report()


    # 3. Ingestar a GCS (y opcionalmente a BigQuery)
    fn_ingesta(fecha_str, var_bucket_name, var_project_id, var_dataset_id, df_cyber)

    # Avanzar un día
    fecha_actual += timedelta(days=1)


INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ffd19ca170>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-01-24&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 15:44:48 GMT
INFO:Authorization:Host: 


📆 Procesando fecha: 2025-01-24
HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_af9e9c68-b446-4084-9e84-cf9f25038b9b-14157-2804', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:44:49 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 15:44:49 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '58e6df2e-b066-40cd-92a9-2de1c07eabd3'})
Download Status :  200
Response Headers :  HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_af9e9c68-b446-4084-9e84-cf9f25038b9b-14157-2804', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:44:49 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'appl

INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ff89cc6260>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-01-25&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 15:45:26 GMT
INFO:Authorization:Host: 

✅ Archivo 'TRR_daily20250123.parquet' guardado en Cloud Storage en 'Raw/Cybersource/Transaccional/TRR_daily/2025/01/TRR_daily20250123.parquet'

📆 Procesando fecha: 2025-01-25
HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_c1741059-9370-42d8-972b-ae31f3331208-15309-9131', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:45:27 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 15:45:27 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '3802d4d2-7a39-4c4e-9e27-84cab3c85522'})
Download Status :  200
Response Headers :  HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_c1741059-9370-42d8-972b-ae31f3331208-15309-9131', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 15:45:2

INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ffd1a72d70>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-01-26&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 15:45:55 GMT
INFO:Authorization:Host: 

...

✅ Archivo 'TRR_daily20250426.parquet' guardado en Cloud Storage en 'Raw/Cybersource/Transaccional/TRR_daily/2025/04/TRR_daily20250426.parquet'

📆 Procesando fecha: 2025-04-28
HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_07f36d88-693b-485b-8a16-15650a0db173-15976-4918', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 16:43:56 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 16:43:56 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '3a69b06f-9fa3-49d5-8efc-4bf52ceff9e0'})
Download Status :  200
Response Headers :  HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_07f36d88-693b-485b-8a16-15650a0db173-15976-4918', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 16:43:5

INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ff5eb94850>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-04-29&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 16:44:23 GMT
INFO:Authorization:Host: 

✅ Archivo 'TRR_daily20250427.parquet' guardado en Cloud Storage en 'Raw/Cybersource/Transaccional/TRR_daily/2025/04/TRR_daily20250427.parquet'

📆 Procesando fecha: 2025-04-29
HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_32f6b668-df06-49c1-995a-b578a6afeb22-38151-11203', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 16:44:23 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 16:44:23 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '1892a2c6-568a-4ee2-b335-15ee78c566aa'})
Download Status :  200
Response Headers :  HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_32f6b668-df06-49c1-995a-b578a6afeb22-38151-11203', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 16:44

INFO:MerchantConfiguration:START> ======================================= 
INFO:MerchantConfiguration:Mconfig >      {'authentication_type': 'http_signature', 'run_environment': 'api.cybersource.com', 'request_json_path': '', 'key_file_name': 'izipay_master_acct', 'keys_directory': '/content/resources', 'use_metakey': 'False', 'portfolio_id': '', 'timeout': '1000', 'useMLEGlobally': 'False', 'log_config': '<CyberSource.logging.log_configuration.LogConfiguration object at 0x77ff89a82bc0>'}
INFO:ReportDownloadsApi:CALL TO METHOD `download_report` STARTED
INFO:Authorization:Using Request Target:   /reporting/v3/report-downloads?organizationId=izipay_master_acct&reportDate=2025-04-30&reportName=TRR_daily
INFO:Authorization:Authentication Type:   http_signature
INFO:Authorization:Request-Type:      GET
INFO:Authorization:Content-Type:   application/json
INFO:Authorization:v-c-merchant-id:   izipay_master_acct
INFO:Authorization:Date:   Mon, 12 May 2025 16:45:06 GMT
INFO:Authorization:Host: 

✅ Archivo 'TRR_daily20250428.parquet' guardado en Cloud Storage en 'Raw/Cybersource/Transaccional/TRR_daily/2025/04/TRR_daily20250428.parquet'

📆 Procesando fecha: 2025-04-30
HTTPHeaderDict({'Cache-Control': 'no-cache, no-transform', 'X-OPNET-Transaction-Trace': 'a2_1725fc58-31e3-4e1c-9c76-669709d591de-14220-4931', 'ETag': '"0"', 'Expires': 'Mon, 12 May 2025 16:45:07 GMT', 'Last-Modified': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/xml', 'Date': 'Mon, 12 May 2025 16:45:07 GMT', 'Server': 'secretServer', 'Strict-Transport-Security': 'max-age=31536000', 'Connection': 'keep-alive', 'transfer-encoding': 'chunked', 'v-c-correlation-id': '509f8255-fc25-4d0c-b9d2-740683e998ee'})
